In [1]:
def makeBoundaries(epsilon, minValue, maxValue):
    m = epsilon * np.ceil(float(1/epsilon) * minValue)
    M = epsilon * np.floor(float(1/epsilon) * maxValue)
    return m, M

def frange(start,stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def MakeSensorCombinations(start, end, epsilon, sensorType, room):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])    
    Xs = frange(a1, b1, epsilon)
    Ys = frange(a2, b2, epsilon)
    
    points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
    C = itertools.combinations(points, distribution[room][types.index(sensorType)])

    return C


def FullySensorizeRooms(epsilon, rooms, types, distribution):
    sensorPlacesByRooms = {}
    sensorsCount = 0
    for room in rooms:
        sensorPlacesByRooms[room] = {}
        for sensorType in range(len(types)):
            points = SensorizeRoom(rooms[room][0], rooms[room][1], epsilon, types[sensorType], room, distribution[room][sensorType])                
            sensorPlacesByRooms[room][types[sensorType][0]] = points
            sensorsCount += len(points)
            

    # print(sensorPlacesByRooms)

    columns = []
    for room in rooms:
        for sensorType in range(len(types)):
            columns.append(room + ":" + types[sensorType][0])
    
    df_ = pd.DataFrame()
    
    for room in rooms:
        for sensorType in sensorPlacesByRooms[room]:
            combinations = sensorPlacesByRooms[room][sensorType]
            tempdf = pd.DataFrame(columns=[room + ":" + sensorType])
            tempdf[room + ":" + sensorType] = [combination for combination in combinations]
            
            df_ = pd.concat([df_,tempdf], ignore_index=True, axis=1)
        
    return df_, sensorsCount / len(types), sensorPlacesByRooms
    
def SensorizeRoom(start, end, epsilon, sensorType, room, count):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])  


    points = []
    
    from random import randrange

    for i in range(count):
      X = randrange(a1 * 10, b1 * 10) / 10
      Y = randrange(a2 * 10, b2 * 10) / 10
      points.append((X, Y))

    points = list(itertools.product(points, [room], [sensorType[0]])) 

    return points

def ConvertE_ToDict(df_, rooms, distribution, types, roomsList, sensorPlaces):
    dictE = {}
    for sensorType in range(len(types)):
        prev = 0
        dictE[types[sensorType][0]] = {}
        A = df_[types[sensorType][0]].to_numpy()
        
        E = np.empty([len(A), len(A[0])])
        for i in range(len(A)):
            E[i] = np.array(A[i])
        
        for roomIndex in range(len(roomsList)):
            dictE[types[sensorType][0]][roomsList[roomIndex]] = []
            sensorsNum = len(sensorPlaces[roomsList[roomIndex]][types[sensorType][0]])
            sensorsNum = sensorsNum + prev
            dictE[types[sensorType][0]][roomsList[roomIndex]] = E[:, prev : sensorsNum]
            prev = sensorsNum
                        
    return dictE

def DeploySimulation():
    simworldname = './Data/simulationWorld2.xml'
    space, rooms = pf.ParseWorld(simworldname)
    types, distribution = pf.GetUsersParameters()

    roomsList = []

    for room in distribution:
        roomsList.append(room)
        
    epsilon = 1
    iteration = 1

    # SCS = MakeSensorConfigurationCombinations(epsilon)
    SCS, allSensorPositionsCount, sensorPlaces = FullySensorizeRooms(epsilon, rooms, types)
    
    # Args = []
    # for c in range(len(SCS.columns)):
        # Args.append((SCS.loc[SCS[c].notnull(), SCS.columns[c]]))

    # args = []
    # for i in range(len(Args)):
    #     for j in range(len(Args[i])):
    #         args.append(Args[i][j])

    # args = tuple(args)
    
    import copy
    types2 = copy.deepcopy(types)

    types2[0][1] = int(allSensorPositionsCount)
    types2[1][1] = int(allSensorPositionsCount)
    
    
    iteration = 1
    df_ = sim_sis.RunSimulator(space, rooms, [types2, [args]], True, True, False, iteration)
    
    return space, rooms, types, distribution, roomsList, SCS, allSensorPositionsCount, sensorPlaces, df_
    
    
def CreateSensorEventsDictionary(space, rooms, types, distribution, roomsList, SCS, sensorPlaces, df_):
    dictE = ConvertE_ToDict(df_, rooms, distribution, types, roomsList, sensorPlaces)
    return dictE

def CreateRoomCombinations(roomX, roomName):
    brTemp = []
    brCombinations = {}
    for i in range(len(roomX)):
        brTemp.append(itertools.combinations(range(len(roomX[i][0])), distribution[roomName][0]))

    for i in range(len(brTemp)):
        if i == 0: 
            brCombinations['motion sensors'] = []
            for c in brTemp[i]:
                brCombinations['motion sensors'].append(c)

        elif i == 1: 
            brCombinations['beacon sensors'] = []
            for c in brTemp[i]:
                brCombinations['beacon sensors'].append(c)

    return brCombinations

def EvaluateConfigurations(dictE, r1Name, r2Name, r1Combinations, r2Combinations, sensorName):
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde

    inter = len(r1Combinations[sensorName])
    taskNum = (len(r1Combinations[sensorName]) * len(r2Combinations[sensorName]))
    for x, y in itertools.product(range(len(r1Combinations[sensorName])), range(len(r2Combinations[sensorName]))):
        if x % 100 == 0:
            sys.stderr.write('\r %.2f percent completed' % float((x * 100)/(inter)))

        pde.ConfigurationEvaluation([(dictE[sensorName][r1Name][:,r1Combinations[sensorName][x]],  
                                      dictE[sensorName][r2Name][:,r2Combinations[sensorName][y]])]) 

    print("Done!")

In [2]:
if __name__ ==  '__main__':
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    Data_path = 'gdrive/Shared drives/Smart Condo - Model-Driven Framework for Sensor Simulation/Codes/Server Side Codes/Version 2.0/'

    import sys
    sys.path.append('gdrive/Shared drives/Smart Condo - Model-Driven Framework for Sensor Simulation/Codes/Server Side Codes/Version 2.0/')

    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde

    simworldname = Data_path + '/Data/simulationWorld2.xml'
    space, rooms = pf.ParseWorld(simworldname)
    types, distribution = pf.GetUsersParameters()


    # print(distribution)

    roomsList = []

    for room in distribution:
        roomsList.append(room)
        
    epsilon = 1
    iteration = 1

    # SCS = MakeSensorConfigurationCombinations(epsilon)
    SCS, allSensorPositionsCount, sensorPlaces = FullySensorizeRooms(epsilon, rooms, types, distribution)
    
    
    import copy
    types2 = copy.deepcopy(types)

    types2[0][1] = int(allSensorPositionsCount)
    types2[1][1] = int(allSensorPositionsCount)
    
    Args = []
    for c in range(len(SCS.columns)):
        Args.append((SCS.loc[SCS[c].notnull(), SCS.columns[c]]))

    args = []
    for i in range(len(Args)):
        for j in range(len(Args[i])):
            args.append(Args[i][j])

    args = tuple(args)

    

    iteration = 1
    df_ = sim_sis.RunSimulator(space, rooms, [types2, [args]], True, True, False, iteration, Data_path)
    


    # return space, rooms, types, distribution, roomsList, SCS, allSensorPositionsCount, sensorPlaces, df_
    
    
    
    
    
    
    

Mounted at /content/gdrive
            time  Agent         x         y      z    Action
0     1499181840      1  0.940000  9.342283  0.505   Prepare
1     1499181843      1  0.940000  9.342283  0.505   Prepare
2     1499181846      1  0.940000  9.342283  0.505   Prepare
3     1499181849      1  0.940000  9.342283  0.505   Prepare
4     1499181852      1  0.940000  9.342283  0.505   Prepare
...          ...    ...       ...       ...    ...       ...
2456  1499189208      1  3.117769  3.240000  0.505  Watch TV
2457  1499189211      1  3.117769  3.240000  0.505  Watch TV
2458  1499189214      1  3.117769  3.240000  0.505  Watch TV
2459  1499189217      1  3.117769  3.240000  0.505  Watch TV
2460  1499189220      1  3.117769  3.240000  0.505  Watch TV

[2461 rows x 6 columns]


In [5]:
df_.to_csv(Data_path + "AR_TEST")